In [2]:
# !apt install git-lfs
!pip install -q transformers
!pip install -q sentencepiece
!pip install -q rouge_score
!pip install -q accelerate -U
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.3 MB/s eta 0:00:00


In [3]:
import pandas as pd

In [5]:
ds = pd.read_csv('concated_full.csv')
ds

,True,Procesing
0,Приветствую вас в курсе в котором мы с вами на...,• Язык Java является популярным и востребованн...
1,Привет В этом уроке мы с вами научимся отслежи...,• В этом уроке мы научимся отслеживать соприко...
2,Привет Хочу презентовать Вам большую онлайн пр...,• Онлайн-программа обучения Java-разработчику ...
3,Давайте решим задачу про комплексную частицу н...,• Комплексная частица состоит из центрального ...
4,Давайте решим задачу угадайку нам известно что...,• Задача угадайку: определить содержимое проби...
...,...,...
797,Давайте сделаем такую простенькую табличку. З...,"• Условная начальная форма, дательный падеж и ..."
798,Перед нами задача на язык Котел. То есть у нас...,• Задача на язык Котел.• У нас есть только чет...
799,"Перед нами селькубская задача, и я не стала пе...",• Исчисляемые формы образуются с помощью трех ...
800,в смысле материал. Тут у нас есть какой-то ве...,• Вещество без границ и определенного размера....


In [6]:
import numpy as np
import pandas as pd
from tqdm import tqdm
# from datasets import Dataset
import datasets
import nltk
from transformers import AutoTokenizer, T5TokenizerFast, Seq2SeqTr  ainingArguments, Seq2SeqTrainer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
import torch
from transformers.optimization import Adafactor, AdafactorSchedule
from sklearn.model_selection import train_test_split
# from datasets import load_dataset, load_metric


metric = datasets.load_metric("rouge")
nltk.download('punkt')

REPO = "t5-russian-summarization"  # Введите наазвание название репозитория
MODEL_NAME = "UrukHan/t5-russian-summarization" # Введите наазвание выбранной модели из хаба
MAX_INPUT = 256  # Введите максимальную длинну входных данных  в токенах (длинна входных фраз в словах (можно считать полслова токен))
MAX_OUTPUT  = 64  # Введите максимальную длинну прогнозов в токенах (можно уменьшить для задач суммризации или других задач где выход короче)
BATCH_SIZE = 8


# Загрузка модели и токенизатора
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

model.config.max_length = MAX_OUTPUT  # по умолчанию 20, поэтому во всех моделях прогнозы обрезаются выходные последовательности


<ipython-input-6-beaa1428af47>:14: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = datasets.load_metric("rouge")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


tokenizer_config.json:   0%|          | 0.00/2.37k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.00M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.63M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.45k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

In [7]:
ds, ds_val = train_test_split(ds, test_size=0.1)

In [8]:
from torch.utils.data import Dataset, DataLoader
from transformers import T5Tokenizer

class SummarizationDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_input_length=1024, max_target_length=200):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_input_length = max_input_length
        self.max_target_length = max_target_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = self.data.iloc[idx]['True']
        summary = self.data.iloc[idx]['Procesing']

        # Tokenize input and target sequences
        inputs = self.tokenizer.encode_plus(
            text,
            max_length=self.max_input_length,
            return_tensors='pt',
            padding='max_length',
            truncation=True
        ) # TODO: maybe add 'summarize:'

        targets = self.tokenizer.encode(
            summary,
            max_length=self.max_target_length,
            return_tensors='pt',
            padding='max_length',
            truncation=True
        )

        return {
            'input_ids': inputs['input_ids'].squeeze(),
            'attention_mask': inputs['attention_mask'].squeeze(),
            'labels': targets.squeeze()
        }



train_dataset = SummarizationDataset(ds, tokenizer)
val_dataset = SummarizationDataset(ds_val, tokenizer)

train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=2, shuffle=False)

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.train().to(device)
optimizer = torch.optim.AdamW(params=model.parameters(), lr=1e-5)

In [10]:
n_epochs = 20

train_loss = []
val_loss = []
for epoch in tqdm(range(n_epochs)):
    train_loss_epoch = []
    val_loss_epoch = []
    # iterate through each batch of the train data
    for i, batch in enumerate(tqdm(train_dataloader)):
        optimizer.zero_grad()

        batch = { k: v.to(device) for k, v in batch.items() }

        outputs = model(**batch)

        loss = outputs[0]
        loss.backward()

        train_loss_epoch.append(loss.item())
        optimizer.step()

    with torch.no_grad():
      for i, batch in enumerate(tqdm(val_dataloader)):
        batch = { k: v.to(device) for k, v in batch.items() }
        outputs = model(**batch)

        loss = outputs[0]

        val_loss_epoch.append(loss.item())

    train_loss_epoch, val_loss_epoch = sum(train_loss_epoch)/len(train_loss_epoch), sum(val_loss_epoch)/len(val_loss_epoch)
    print(f'Epoch {epoch}. Train Loss: {train_loss_epoch}. Val Loss: {val_loss_epoch}')
    train_loss.append(train_loss_epoch)
    train_loss.append(val_loss_epoch)

100%|██████████| 361/361 [04:25<00:00,  1.36it/s]

  5%|▌         | 1/20 [04:36<1:27:37, 276.73s/it]

Epoch 0. Train Loss: 1.8287098936096784. Val Loss: 1.3158785628109444



100%|██████████| 361/361 [04:22<00:00,  1.37it/s]

 10%|█         | 2/20 [09:11<1:22:36, 275.34s/it]

Epoch 1. Train Loss: 1.1317419947184355. Val Loss: 1.1945872713879842



100%|██████████| 361/361 [04:22<00:00,  1.37it/s]

 15%|█▌        | 3/20 [13:45<1:17:52, 274.86s/it]

...

Epoch 17. Train Loss: 0.5245146800936754. Val Loss: 1.0282670637456381



100%|██████████| 361/361 [04:22<00:00,  1.37it/s]

 95%|█████████▌| 19/20 [1:26:56<04:34, 274.32s/it]

Epoch 18. Train Loss: 0.5059533205058766. Val Loss: 1.027734834973405



100%|██████████| 361/361 [04:22<00:00,  1.37it/s]

100%|██████████| 20/20 [1:31:30<00:00, 274.53s/it]

Epoch 19. Train Loss: 0.4889121096467708. Val Loss: 1.0228597071112655


In [11]:
summary_ids = model.generate(torch.tensor(tokenizer(ds_val.iloc[10, 0])['input_ids']).reshape(1, -1).to(device),

                                    num_beams=int(2),

                                    no_repeat_ngram_size=3,

                                    length_penalty=2.0,

                                    min_length=2,

                                    max_length=100,

                                    early_stopping=True)

In [12]:
output = tokenizer.decode(summary_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
output

' В математике можно нарисовать кривую, можно использовать как причастие. Первая группа новобрачных - существительное, подлежащее. Вторая группа - принадлежащее, которое не может быть подлежащим. Первое существительное - это приданое, дается в первой группе. Приданое с двумя Н. - часовой механизм может быть использован для решения задачи.'

In [13]:
ds_val.iloc[10, 0]

' В, например, математики нарисовать кривую. но это может быть и прилагательным. Та же самая кривая линия. Мы можем уточнять, можем не уточнять. Значит, это вторая группа Легкая. Это орган в человеческом теле. Но при этом можно сказать легкое прикосновение, легкое дыхание, то есть использовать как прилагательное. Это вторая группа новобрачные. Мы не говорим новобрачные. Люди новобрачные. молодожены. Просто говорим новобрачные. Значит, это не может быть прилагательным, это только существительное. Первая группа подлежащее. Тут хочется сказать, что тоже не бывает подлежащее слово или подлежащее, часть речи или еще что-нибудь такое. И хочется это отправить тоже в первую группу. Но мы ведь можем сказать требования, подлежащие неукоснительному исполнению. то есть использовать его в причастной функции. Так что это будет все-таки вторая группа.  Приданое. Тут тоже такой ловушечный момент, потому что есть слово приданое с двумя Н. и оно как раз будет во второй группе. А вот при данной с одной Н

In [14]:
import pickle
output_file = 'summmmodel.pkl'
with open(output_file, 'wb') as file:
    pickle.dump(model, file)
